# Scraping

Le scraping consiste à extraire des données de pages Web de manière automatique.

La méthode la plus simple consiste à demander le code HTML de la page désirée et à utiliser sa structure pour accéder aux données intéressantes. Ces données peuvent contenir des liens vers d’autres pages à scraper.

In [19]:
# Ce module permet de faire des «requêtes»: donne-moi telle page Web
import requests

# Ce module sert à parcourir la structure d ela page
from bs4 import BeautifulSoup

In [15]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [22]:
# Ici, on envoie une requête pour la page
response = requests.get('https://www.arcinfo.ch')

SSLError: HTTPSConnectionPool(host='www.arcinfo.ch', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))

In [9]:
# Quel est le statut de la réponse? Si c’est un 404, la page n’a pas été trouvée
response

<Response [200]>

In [10]:
# On charge la page dans BeautifulSoup
doc = BeautifulSoup(response.text)

## Parcourir la structure de la page
A partir de cette étape, la structure de la page est contenue dans notre variable `doc`.

HTML est structuré par des balises, des mots-clés entourés des signes < et >:

* un paragraphe est signalé par les balises `<p>` (au début) et `</p>` (à la fin)
* le titre principal est entre les balises `<h1>` et `</h1>` (header 1)
* un tableau est entre les balises `<table>` … `</table>`
* une ligne de tableau est entre les balises `<tr>` … `</tr>` (table row)
* une cellule de tableau est entre les balises `<td>` … `</td>` (table data)

Nous pouvons demander à BeautifulSoup de nous chercher des balises HTML avec les fonctions:

`find`: trouver la première balise correspondante

`find_all`: trouver toutes les balises correspondantes

In [11]:
link = doc.find('a')
link

<a href="/" style="display: none;" title="YouTube"><svg class="external-icon" height="18" id="yt-ringo2-svg" viewbox="0 0 104 18" width="104" xmlns="http://www.w3.org/2000/svg"><defs><clippath id="clip0_161_5084"><rect height="18" width="104"></rect></clippath></defs><g clip-path="url(#clip0_161_5084)"><path d="M13.0363 18C13.0363 18 21.2125 18 23.2406 17.46C24.3825 17.154 25.2413 16.272 25.5427 15.183C26.1 13.185 26.1 8.982 26.1 8.982C26.1 8.982 26.1 4.806 25.5427 2.826C25.2413 1.71 24.3825 0.846 23.2406 0.549C21.2125 0 13.0363 0 13.0363 0C13.0363 0 4.87833 0 2.8594 0.549C1.73574 0.846 0.858733 1.71 0.538992 2.826C0 4.806 0 8.982 0 8.982C0 8.982 0 13.185 0.538992 15.183C0.858733 16.272 1.73574 17.154 2.8594 17.46C4.87833 18 13.0363 18 13.0363 18Z" fill="#FF0000"></path><path d="M17.1 8.99999L10.35 5.17499V12.825L17.1 8.99999Z" fill="white"></path><path d="M104 0V18" stroke="grey" stroke-width="0.5"></path><g id="youtube-paths"><path d="M34.2685 18.0201V12.4901L37.7785 1.00012H34.9185L

In [12]:
links = doc.find_all('a')

# Cela m’a donné 32 liens!
len(links)

15

In [13]:
p_list = doc.find_all('p')
len(p_list)

0

## A vous

Essayez d’autres balises, par exemple:

* doc.find_all('p')
* doc.find_all('img')
* doc.find_all('span')

## Trouver les attributs
Trouver des liens, c’est bien. Mais comment extraire l’URL vers lequel ils conduisent?

La méthode `.get()` est faite pour ça: elle récupère les attributs d’une balise HTML, comme l’attribut «href» qui contient ces URL.

In [8]:
links[6].get('href')

'http://paulronga.ch/#portfolio'

## … et avec une boucle
Mettons que nous voulons récupérer toutes les URL de notre menu:
* avec une boucle `for`, on peut passer les liens en revue
* la méthode `.get()` nous permettra de récupérer l’URL de chaque lien

In [9]:
for a in doc.select('.nav-menu a'):
    print(a.get('href'))

http://paulronga.ch
http://paulronga.ch/#about
http://paulronga.ch/#video
http://paulronga.ch/#portfolio
http://paulronga.ch/#news


## Selecteurs
Les balises html contiennent parfois les attributs **id** et/ou **class**. Concrètement, ça se présente par exemple comme ça:

`<table id="resultats" class="bordered striped">`

L’attribut **id** (identifiant) est unique: un seul élément le porte.

L’attribut **class** (classes) peut être ajouté à plusieurs éléments.

Les classes et identifiants servent à styliser la page: tels éléments ont une bordure rouge, un fond gris clair, etc. Ces instructions sont regroupées dans du [code CSS](https://www.w3.org/Style/Examples/011/firstcss.fr.html), un langage informatique qui définit la présentation du HTML.

Dans le code CSS, on les éléments du HTML par des **sélecteurs**. En voici des exemples:

`table
#resultats
table.striped
.bordered`

Tous ces sélecteurs renvoient à la `<table id="resultats" class="bordered striped">` donnée comme exemple plus haut. On peut donc cibler une balise HTML par son nom, par son id (en le précédent d’un "#") ou par une de ses classes.

Pour trouver le sélecteur d’un élément qui vous intéresse, vous pouvez inspecter la page (cmd-alt-i dans Firefox ou Chrome), le repérer dans «Elements» grâce à la flèche tout à gauche, puis faire clic droit -> Copy -> CSS selector.

Avec BeautifulSoup, on peut retrouver les éléments correspondants à un sélecteur grâce à la méthode:

`.select('selecteur')`

Cette méthode renvoie une liste, voici un exemple:

In [10]:
doc.select('.nav-menu a')

[<a aria-current="page" href="http://paulronga.ch">Accueil</a>,
 <a aria-current="page" href="http://paulronga.ch/#about">Le journalisme data</a>,
 <a aria-current="page" href="http://paulronga.ch/#video">Vidéo</a>,
 <a aria-current="page" href="http://paulronga.ch/#portfolio">Portfolio</a>,
 <a aria-current="page" href="http://paulronga.ch/#news">Blog</a>]

# Attributs et méthodes

Mettez doc.find('p') dans la variable `test`, puis écrivez `test.` et appuyez sur tab en laissant votre curseur après le point. Quels attributs et méthodes apparaissent? Essayez-en quelques-uns.

Vous avez trouvez? Allez, je vous en donne trois:
* `.string` donne le texte de la balise uniquement
* `.text` donne le texte de la balise et de toutes les balises qu’elle contient
* `.get()` permet d’obtenir un attribut précis, comme id, class, href

Pour avoir des exemples clairs, on va charger une page spécialement préparée pour votre scraping.

In [11]:
response = requests.get('https://exemple.tcch.ch/scraping/')
doc = BeautifulSoup(response.content, 'html.parser')

In [12]:
# premier paragraphe
p = doc.find('p')
print('La balise et son contenu:', p)
print('Le contenu grâce à .string:', p.string)
print('Le contenu grâce à .text:', p.text)

La balise et son contenu: <p>Ce paragraphe nous intéresse. C’est le premier.</p>
Le contenu grâce à .string: Ce paragraphe nous intéresse. C’est le premier.
Le contenu grâce à .text: Ce paragraphe nous intéresse. C’est le premier.


Quelle différence entre `.string` et `.text`? On le comprend quand notre balise contient d’autres balises. C’est le cas du dernier paragraphe: il contient une balise `<em>`.

In [13]:
# dernier paragraphe
p = doc.select('p')[-1]
p

<p>Le troisième est un peu plus <em>compliqué</em>…<br/>
    Mais on va s’en sortir!</p>

In [14]:
print('.string:', p.string)
print('.text:', p.text)

.string: None
.text: Le troisième est un peu plus compliqué…
    Mais on va s’en sortir!


Quand une balise contient d’autres balises, `.string` renvoie donc **None**, tandis que `.text` renvoie tout le texte contenu dans la balise principale et les balises qu’elle contient.

## A vous
Qu’est-ce que `.string` et `.text` donnent comme résultat avec la table? Essayez!

Le système que nous avons vu ne fonctionne pas pour certaines pages Web construites dynamiquement avec du JavaScript. Dans ce cas, on peut automatiser un navigateur tel que Chrome ou Firefox avec des outils comme [Selenium](https://pypi.org/project/selenium/). C’est nettement plus complexe que d’utiliser requests et BeautifulSoup.